In [2]:
import pandas as pd
import glob
import os
import json
import numpy as np
from  cycler import cycler
import fastplot
from collections import Counter, defaultdict
from statistics import mode
import statsmodels.stats.proportion
import re

CYCLER_LINESPOINTS=(cycler('color', ['r', 'b', 'g', 'purple', 'c']) +
                    cycler('linestyle', ['-', '--', '-.', ':', (0, (3, 1, 1, 1)) ]) +
                    cycler('marker', ['o', 's', 'v', 'd', '^' ]))

CYCLER_LINES=(cycler('color', ['r', 'b', 'g', 'purple', 'c']) +
                    cycler('linestyle', ['-', '--', '-.', ':', (0, (3, 1, 1, 1)) ]))

PLOT_ARGS = {"style":"latex", "grid":True, "figsize":(4.25,2.5), "rcParams":{'text.latex.preamble': '\\usepackage{libertine}\n\\usepackage{setspace}'}, "legend_args":{"columnspacing":1, "handlelength":1.5}}
PLOT_ARGS_LARGE = {"style":"latex", "grid":True, "figsize":(9,2.75), "rcParams":{'text.latex.preamble': '\\usepackage{libertine}\n\\usepackage{setspace}'}, "legend_args":{"columnspacing":1, "handlelength":1.5}}

BEFORE_STR = "\\emph{Before-Accept}"
AFTER_STR = "\\emph{After-Accept}"
ADDITIONAL_STR = "\\emph{Additional-Visits}"

TRANSLATE_CATEGORIES = {"Computers Electronics and Technology": "Computers and Technology",
                        "Business and Consumer Services": "Business and Services",
                        "Heavy Industry and Engineering": "Industry and Engineering"}

COUNTRY_TRAD={"it":"Italy", "fr":"France", "de":"Germany", "es":"Spain", "uk":"UK", "us": "US"}

In [3]:
trackers = set( open("trackers-list.txt", "r").read().splitlines() )

similarweb_websites=defaultdict(set)
similarweb_websites['us'] = set(open("similarweb/USA.txt", "r" ).read().splitlines()) 
    

In [4]:
bad_domains=set("co.uk co.jp co.hu co.il com.au co.ve .co.in com.ec com.pk co.th co.nz com.br com.sg com.sa \
com.do co.za com.hk com.mx com.ly com.ua com.eg com.pe com.tr co.kr com.ng com.pe com.pk co.th \
com.au com.ph com.my com.tw com.ec com.kw co.in co.id com.com com.vn com.bd com.ar \
com.co com.vn org.uk net.gr".split())

def getGood2LD(fqdn):
    if fqdn[-1] == ".":
        fqdn = fqdn[:-1]    
    names = fqdn.split(".")
    if ".".join(names[-2:]) in bad_domains:
        return get3LD(fqdn)
    tln_array = names[-2:]
    tln = ""
    for s in tln_array:
        tln = tln + "." + s
    return tln[1:]

def get3LD(fqdn):
    if fqdn[-1] == ".":
        fqdn = fqdn[:-1]
    names = fqdn.split(".")
    tln_array = names[-3:]
    tln = ""
    for s in tln_array:
        tln = tln + "." + s
    return tln[1:]

In [5]:
df = pd.read_csv("../output.csv")

In [6]:
print(df.iloc[0])

url                                               http://character.ai
landing_page                                    https://character.ai/
has_found_banner                                                False
cookie_first        [{"domain": "www.google.com", "expires": 17249...
cookie_click        [{"domain": "www.google.com", "expires": 17249...
cookie_internal     [{"domain": "www.google.com", "expires": 17249...
domains_first       ["events.character.ai", "www.gstatic.com", "st...
domains_click       ["character.ai", "o4504695552606208.ingest.sen...
Name: 0, dtype: object


In [7]:
df["sld"] = df["url"].apply(lambda u: getGood2LD(u.split("/")[2] ).strip() ) 
df["tld"] = df["url"].apply(lambda u: get3LD(u.split("/")[2] ).strip()  ) 

In [8]:
print(df.iloc[0])

url                                               http://character.ai
landing_page                                    https://character.ai/
has_found_banner                                                False
cookie_first        [{"domain": "www.google.com", "expires": 17249...
cookie_click        [{"domain": "www.google.com", "expires": 17249...
cookie_internal     [{"domain": "www.google.com", "expires": 17249...
domains_first       ["events.character.ai", "www.gstatic.com", "st...
domains_click       ["character.ai", "o4504695552606208.ingest.sen...
sld                                                      character.ai
tld                                                      character.ai
Name: 0, dtype: object


In [9]:
df['us'] = True
print(df.iloc[0])

url                                               http://character.ai
landing_page                                    https://character.ai/
has_found_banner                                                False
cookie_first        [{"domain": "www.google.com", "expires": 17249...
cookie_click        [{"domain": "www.google.com", "expires": 17249...
cookie_internal     [{"domain": "www.google.com", "expires": 17249...
domains_first       ["events.character.ai", "www.gstatic.com", "st...
domains_click       ["character.ai", "o4504695552606208.ingest.sen...
sld                                                      character.ai
tld                                                      character.ai
us                                                               True
Name: 0, dtype: object


In [10]:
similarweb_csv = pd.read_csv("similarweb/together.csv")

In [11]:
# adding categories to the dataframe
similarweb_csv["category"] = similarweb_csv["category"].apply(lambda s: TRANSLATE_CATEGORIES[s] if s in TRANSLATE_CATEGORIES else s)
categories = defaultdict(set)

for row in similarweb_csv.itertuples():
    if row.category != "ALL" and row.country in {"France", "USA", "UK"} :
        categories[row.category].add(row.website)
        
for category in categories:
    df[category] = (df.sld.isin(categories[category])) | (df.tld.isin(categories[category]))


In [12]:
print(df.iloc[0])

url                                                                   http://character.ai
landing_page                                                        https://character.ai/
has_found_banner                                                                    False
cookie_first                            [{"domain": "www.google.com", "expires": 17249...
cookie_click                            [{"domain": "www.google.com", "expires": 17249...
cookie_internal                         [{"domain": "www.google.com", "expires": 17249...
domains_first                           ["events.character.ai", "www.gstatic.com", "st...
domains_click                           ["character.ai", "o4504695552606208.ingest.sen...
sld                                                                          character.ai
tld                                                                          character.ai
us                                                                                   True
Adult     

In [13]:
# getting the trackers for before, after and internal banner.
# Note that only checking the tracker that is in the tracker list and not expried
def get_trackers(s):
    d = json.loads(s)
    this_trackers = set()
    for c in d:
        if (c["domain"] in trackers or getGood2LD(c["domain"]) in trackers or get3LD(c["domain"]) in trackers) and c["expires"] > 0:
            this_trackers.add(getGood2LD(c["domain"]))
    return list(this_trackers)

df["trackers_first"] = df["cookie_first"].apply(get_trackers)
df["trackers_click"] = df["cookie_click"].apply(get_trackers)
df["trackers_internal"] = df["cookie_internal"].apply(get_trackers)

In [14]:
print(df.iloc[0])

url                                                                   http://character.ai
landing_page                                                        https://character.ai/
has_found_banner                                                                    False
cookie_first                            [{"domain": "www.google.com", "expires": 17249...
cookie_click                            [{"domain": "www.google.com", "expires": 17249...
cookie_internal                         [{"domain": "www.google.com", "expires": 17249...
domains_first                           ["events.character.ai", "www.gstatic.com", "st...
domains_click                           ["character.ai", "o4504695552606208.ingest.sen...
sld                                                                          character.ai
tld                                                                          character.ai
us                                                                                   True
Adult     

In [15]:

def website_stats(this_df):
    num = len(this_df.index)
    trackers_first = set()
    trackers_click = set()
    trackers_internal = set()
    
    trackers_first_all = set(this_df.iloc[0].trackers_first)
    trackers_click_all = set(this_df.iloc[0].trackers_click)
    trackers_internal_all = set(this_df.iloc[0].trackers_internal)
    trackers_details = []

    has_found_banner = False
    
    tp_first = set()
    tp_click = set()
    
    countries = { c:False for c in similarweb_websites.keys()}
    cats = { c:False for c in categories.keys()}
    #+ list(similarweb_websites.keys()) + list(categories.keys())
    
    
    for i, row in this_df.iterrows():

        trackers_first |= set(row.trackers_first)
        trackers_click |= set(row.trackers_click)
        trackers_internal |= set(row.trackers_internal)
        
        trackers_first_all &= set(row.trackers_first)
        trackers_click_all &= set(row.trackers_click)
        trackers_internal_all &= set(row.trackers_internal)
        
        trackers_details.append(set(row.trackers_click))
        
        tp_first |= set([ getGood2LD(d) for d in json.loads(row.domains_first) if d != ""] )
        tp_click |= set([ getGood2LD(d) for d in json.loads(row.domains_click) if d != ""])      
        
        for c in list(similarweb_websites.keys()):
            countries[c] |= row[c]

        for c in list(categories.keys()):
            cats[c] |= row[c]
            
        if row.has_found_banner:
            has_found_banner=True
        
    trackers_only_click = trackers_click - trackers_first
    
    trackers_acc = set()
    trackers_incremental = []
    for s in trackers_details:
        trackers_acc |= s
        trackers_incremental.append(len(trackers_acc))
    
    return pd.Series({"count": num,
                      "trackers_first": trackers_first,
                      "trackers_click": trackers_click,
                      "trackers_only_click": trackers_only_click,
                      "trackers_internal": trackers_internal,
                      
                      "trackers_first_all": trackers_first_all,
                      "trackers_click_all": trackers_click_all,
                      "trackers_internal_all": trackers_internal_all,
                      
                      "trackers_incremental_nb": trackers_incremental,
                    
                      "tp_first": tp_first,
                      "tp_click": tp_click,
                      "has_found_banner": has_found_banner,
                      **countries, **cats
                     })
# grouped = df.groupby(["sld"]).apply(website_stats).reset_index()
grouped = df.groupby(["sld"]).apply(website_stats,include_groups=False).reset_index()
grouped

,sld,count,trackers_first,trackers_click,trackers_only_click,trackers_internal,trackers_first_all,trackers_click_all,trackers_internal_all,trackers_incremental_nb,...,Jobs_and_Career,Law_and_Government,News_and_Media,Pets_and_Animals,Reference_Materials,Science_and_Education,Sports,Travel_and_Tourism,Vehicles,Lifestyle
0,0123movie.net,1,{},{},{},{},{},{},{},[0],...,False,False,False,False,False,False,False,False,False,False
1,123rf.com,1,"{doubleclick.net, quantserve.com, twitter.com,...","{doubleclick.net, quantserve.com, twitter.com,...",{},"{vimeo.com, twitter.com, doubleclick.net, quan...","{doubleclick.net, twitter.com, quantserve.com,...","{doubleclick.net, twitter.com, quantserve.com,...","{twitter.com, vimeo.com, doubleclick.net, tikt...",[4],...,False,False,False,False,False,False,False,False,False,False
2,16personalities.com,1,{},{},{},{},{},{},{},[0],...,True,False,False,False,False,False,False,False,False,False
3,17track.net,1,{},{},{},{},{},{},{},[0],...,False,False,False,False,False,False,False,False,False,False
4,1800petmeds.com,1,"{clarity.ms, agkn.com, turn.com, tiktok.com, d...","{clarity.ms, agkn.com, turn.com, tiktok.com, d...",{},"{clarity.ms, agkn.com, turn.com, tiktok.com, d...","{clarity.ms, agkn.com, turn.com, tiktok.com, d...","{clarity.ms, agkn.com, turn.com, tiktok.com, d...","{clarity.ms, agkn.com, turn.com, tiktok.com, d...",[8],...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,zoosk.com,1,"{adnxs.com, tiktok.com, doubleclick.net, pinte...","{adnxs.com, tiktok.com, doubleclick.net, pinte...","{casalemedia.com, bidswitch.net}","{adnxs.com, tiktok.com, doubleclick.net, pinte...","{adnxs.com, pinterest.com, doubleclick.net, ti...","{adnxs.com, pinterest.com, doubleclick.net, ti...","{adnxs.com, pinterest.com, doubleclick.net, ti...",[12],...,False,False,False,False,False,False,False,False,False,False
2098,zoro.com,1,{},{},{},{},{},{},{},[0],...,False,False,False,False,False,False,False,False,False,False
2099,zorox.to,1,{},{},{},{},{},{},{},[0],...,False,False,False,False,False,False,False,False,False,False
2100,zulacasino.com,1,{},{},{},{},{},{},{},[0],...,False,False,False,False,False,False,False,False,False,False


In [40]:
PLOT_ARGS_SMALL = {"style":"latex", "grid":True, "figsize":(2,2.25), "rcParams":{'text.latex.preamble': r'\usepackage{libertine}'}, "legend_args":{"columnspacing":1, "handlelength":1.5}}

W=100
for c in ["us"]:

    plot_data = grouped [ grouped[[c]].any(axis=1) ].copy()
    missing = 0#10542 - len(plot_data.index)
    plot_data["trackers_first_nb"] = plot_data["trackers_first"].apply(len)
    plot_data["trackers_click_nb"] = plot_data["trackers_click"].apply(len)
    plot_data = plot_data[["trackers_first_nb", "trackers_click_nb"]]

    # plot_data = plot_data.append( [{"trackers_first_nb":0, "trackers_click_nb":0}]*missing ).sample(frac=1)
    plot_data = plot_data._append( [{"trackers_first_nb":0, "trackers_click_nb":0}]*missing ).sample(frac=1)

    plot_data= plot_data.sort_values(by="trackers_first_nb", ascending=False).reset_index()
    plot_list = [(BEFORE_STR,   (np.arange(len(plot_data["trackers_first_nb"].values ) ) +1 ,   plot_data["trackers_first_nb"].values  ) ),
                 (AFTER_STR, (np.arange(len(plot_data["trackers_click_nb"].values) ) +1 , plot_data["trackers_click_nb"].values ) ),
                ]

    avg = (np.arange(len(plot_data["trackers_click_nb"].values) ) +1 , plot_data["trackers_click_nb"].rolling(W).mean().values )
        
    CYCLER_LINESANDPOINTS=(cycler('color', ['r', 'b', 'b', 'b', 'c']) +
                        cycler('linestyle', ['-', '', '-.', '-', (0, (3, 1, 1, 1)) ]) +
                        cycler('marker', ['', 's', '', '', '^' ]))

    has_legend = c == "fr"
    ylabel = 'Tracker Number' if has_legend else None
    yticks = ([0,20,40,60,80],None) if has_legend else ([0,20,40,60,80],["","","","",""])
    #xlabel = "{}\\\\Websites".format(COUNTRY_TRAD[c]) if c == "it" else "{}".format(COUNTRY_TRAD[c])
    xlabel = "{}".format(COUNTRY_TRAD[c])
    
    plot = fastplot.plot( plot_list ,None, 
                   mode='line_multi', ylabel = ylabel,
                   legend = has_legend, legend_loc='upper right', legend_ncol=1, legend_fontsize=12, legend_alpha=0.5,
                     xlabel = "Websites", linewidth=2, fontsize=12,
                   xlim = (-100, 2500), cycler = CYCLER_LINESANDPOINTS, plot_args = {"markersize": 1},
                         xticks = ([0,1000,2000],None), yticks = yticks,
                   ylim = (-5,80), **PLOT_ARGS_SMALL)

    if has_legend:
        # plot.gca().get_legend().legendHandles[1]._legmarker.set_markersize(5)
        plot.gca().get_legend().legend_handles[1].set_markersize(5)
    plot.plot(plot_list[0][1][0], plot_list[0][1][1], "-", color="red", marker=None, markersize=0)
    plot.title(xlabel)
    
    plot.plot(avg[0], avg[1])
    
    plot.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9) 
    plot.savefig("plots/cookieaccept_overall_tracker_nb_{}.pdf".format(c), bbox_inches = "tight")
    
    plot.show()

C:\Users\zhang\AppData\Local\Temp\ipykernel_876\94732607.py:51: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [41]:
print(similarweb_websites.keys())

dict_keys(['us'])


In [42]:
plot_data = []

for country in similarweb_websites.keys():
    this_grouped = grouped[grouped[country] == True]
    tot = len(this_grouped.index)
    tracker_first = len(this_grouped[this_grouped["trackers_first"].apply(lambda s: len(s) > 0 ) ].index)
    tracker_click = len(this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) > 0 ) ].index)
    tracker_internal = len(this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) > 0 ) ].index)
    plot_data.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR: tracker_first/tot*100,
                      AFTER_STR: tracker_click/tot*100,
                      ADDITIONAL_STR: tracker_internal/tot*100,
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.sort_values(by="country")
plot_df.index = plot_df["country"]
del plot_df["country"]

THIS_PLOT_ARGS = PLOT_ARGS.copy()
THIS_PLOT_ARGS["figsize"] = (4.25,2.25)
    
plot = fastplot.plot( plot_df ,None, 
               mode='bars_multi', ylabel = 'Websites With Trackers [\\%]',
               legend = True, legend_loc='upper left', legend_ncol=3, legend_fontsize="small",
               grid_axis="y",
               ylim = (0,100), **THIS_PLOT_ARGS)


for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch("\\\\\\\\\\")
    p.set_edgecolor ("red")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):len(plot_df)*2]+ [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")
    
for p in plot.gca().patches[2*len(plot_df):] + [plot.gca().get_legend().get_patches()[2]]:
    p.set_hatch("----")
    p.set_edgecolor ("green")
    p.set_facecolor ("white")

#plot.gca().patches[5].set_hatch("XXXX")
#plot.gca().patches[11].set_hatch("++++")
#plot.gca().patches[17].set_hatch("....")

for i, country in enumerate(["us"]):
    this_grouped = grouped[grouped[country] == True]
    n = round(len(this_grouped[this_grouped["has_found_banner"]].index)/len(this_grouped.index)*100,1)
    plot.text(i, 105, "{}\\%".format(n), ha="center", bbox = dict(boxstyle='square,pad=0', color="white"))
    
plot.text(2.5, 115, "Consent Banner Found and Accepted", ha="center", bbox = dict(boxstyle='square,pad=0', facecolor='white', color="white"))

plot.subplots_adjust(left=0.125, bottom=0.1, right=0.95, top=0.85) 

# err = plot_df.applymap(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*2500/100,2500,0.05) ]  )
err = plot_df.map(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*2500/100,2500,0.05) ]  )
for i, tup in enumerate(err.itertuples()):
    plot.gca().plot( (i-0.2,i-0.2), (tup[1]), linestyle="-", color="black"  )
    plot.gca().plot( (i,i), (tup[2]), linestyle="-", color="black" )
    plot.gca().plot( (i+0.2,i+0.2), (tup[3]), linestyle="-", color="black" )
    
plot.savefig("plots/cookieaccept_websites_with_trackers.pdf", bbox_inches = "tight")
plot.show()

KeyError: 'fr'

In [44]:
plot_data = []

for country in similarweb_websites.keys():
    this_grouped = grouped[grouped[country] == True]
    tot = len(this_grouped.index)
    tracker_first = len(this_grouped[this_grouped["trackers_first"].apply(lambda s: len(s) > 0 ) ].index)
    tracker_click = len(this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) > 0 ) ].index)
    tracker_internal = len(this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) > 0 ) ].index)
    plot_data.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR: tracker_first/tot*100,
                      AFTER_STR: tracker_click/tot*100,
                      ADDITIONAL_STR: tracker_internal/tot*100,
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.sort_values(by="country")
plot_df.index = plot_df["country"]
del plot_df["country"]

THIS_PLOT_ARGS = PLOT_ARGS.copy()
THIS_PLOT_ARGS["figsize"] = (4.25,2.25)
    
plot = fastplot.plot( plot_df ,None, 
               mode='bars_multi', ylabel = 'Websites With Trackers [\\%]',
               legend = True, legend_loc='upper left', legend_ncol=3, legend_fontsize="small",
               grid_axis="y",
               ylim = (0,100), **THIS_PLOT_ARGS)


for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch("\\\\\\\\\\")
    p.set_edgecolor ("red")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):len(plot_df)*2]+ [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")
    
for p in plot.gca().patches[2*len(plot_df):] + [plot.gca().get_legend().get_patches()[2]]:
    p.set_hatch("----")
    p.set_edgecolor ("green")
    p.set_facecolor ("white")

#plot.gca().patches[5].set_hatch("XXXX")
#plot.gca().patches[11].set_hatch("++++")
#plot.gca().patches[17].set_hatch("....")

for i, country in enumerate(["us"]):
    this_grouped = grouped[grouped[country] == True]
    n = round(len(this_grouped[this_grouped["has_found_banner"]].index)/len(this_grouped.index)*100,1)
    plot.text(i, 105, "{}\\%".format(n), ha="center", bbox = dict(boxstyle='square,pad=0', color="white"))
    
plot.text(2.5, 115, "Consent Banner Found and Accepted", ha="center", bbox = dict(boxstyle='square,pad=0', facecolor='white', color="white"))

plot.subplots_adjust(left=0.125, bottom=0.1, right=0.95, top=0.85) 

# err = plot_df.applymap(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*2500/100,2500,0.05) ]  )
err = plot_df.map(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*2500/100,2500,0.05) ]  )
for i, tup in enumerate(err.itertuples()):
    plot.gca().plot( (i-0.2,i-0.2), (tup[1]), linestyle="-", color="black"  )
    plot.gca().plot( (i,i), (tup[2]), linestyle="-", color="black" )
    plot.gca().plot( (i+0.2,i+0.2), (tup[3]), linestyle="-", color="black" )
    
plot.savefig("plots/cookieaccept_websites_with_trackers.pdf", bbox_inches = "tight")
plot.show()

C:\Users\zhang\AppData\Local\Temp\ipykernel_876\39002177.py:53: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  plot.text(2.5, 115, "Consent Banner Found and Accepted", ha="center", bbox = dict(boxstyle='square,pad=0', facecolor='white', color="white"))
C:\Users\zhang\AppData\Local\Temp\ipykernel_876\39002177.py:65: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [45]:
plot_data = []

for country in similarweb_websites.keys():
    this_grouped = grouped[grouped[country] == True]
    tot = len(this_grouped.index)
    tot_wo = len(this_grouped[this_grouped["has_found_banner"]==False].index)
    tot_w = len(this_grouped[this_grouped["has_found_banner"]==True].index)
    tracker_first_wo = len(this_grouped[ (this_grouped["trackers_first"].apply(lambda s: len(s) > 0 )) & ( this_grouped["has_found_banner"]==False) ].index)
    tracker_click_wo = len(this_grouped[ (this_grouped["trackers_click"].apply(lambda s: len(s) > 0 )) & ( this_grouped["has_found_banner"]==False) ].index)
    tracker_first_w = len(this_grouped[ (this_grouped["trackers_first"].apply(lambda s: len(s) > 0 )) & ( this_grouped["has_found_banner"]==True) ].index)
    tracker_click_w = len(this_grouped[ (this_grouped["trackers_click"].apply(lambda s: len(s) > 0 )) &( this_grouped["has_found_banner"]==True) ].index)
    plot_data.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR +  " (w/o Banner)": tracker_click_wo/tot_wo*100, # Use the "Click" visits, to take into account all 
                      BEFORE_STR + " (w/ Banner)": tracker_first_w/tot_w*100,
                      AFTER_STR +  " (w/ Banner)": tracker_click_w/tot_w*100,
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.sort_values(by="country")
plot_df.index = plot_df["country"]
del plot_df["country"]

THIS_PLOT_ARGS = PLOT_ARGS.copy()
THIS_PLOT_ARGS["figsize"] = (4.25,2.25)
    
plot = fastplot.plot( plot_df ,None, 
               mode='bars_multi', ylabel = 'Websites With Trackers [\\%]',
               legend = True, legend_loc='upper left', legend_ncol=2, legend_fontsize="small", legend_alpha = 0.5,
               grid_axis="y",
               ylim = (0,120), **THIS_PLOT_ARGS)


for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch(r'|\|\|\|\\')
    p.set_edgecolor ("indianred")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):2*len(plot_df)] + [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch(r'-\-\-\-\\')
    p.set_edgecolor ("darkred")
    p.set_facecolor ("white")

for p in plot.gca().patches[2*len(plot_df):] + [plot.gca().get_legend().get_patches()[2]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")

plot.subplots_adjust(left=0.125, bottom=0.1, right=0.95, top=0.85) 

# err = plot_df.applymap(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*2500/100,2500,0.05) ]  )
err = plot_df.map(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*2500/100,2500,0.05) ]  )
for i, tup in enumerate(err.itertuples()):
    plot.gca().plot( (i-0.2,i-0.2), (tup[1]), linestyle="-", color="black"  )
    plot.gca().plot( (i,i), (tup[2]), linestyle="-", color="black" )
    plot.gca().plot( (i+0.2,i+0.2), (tup[3]), linestyle="-", color="black" )

plot.savefig("plots/cookieaccept_websites_with_trackers_separate.pdf", bbox_inches = "tight")
plot.show()

C:\Users\zhang\AppData\Local\Temp\ipykernel_876\229267521.py:57: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [46]:
plot_data = []
plot_data_error = []
for country in similarweb_websites.keys():
    this_grouped = grouped[grouped[country] == True]
    tracker_first = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_first"].apply(len).mean()
    tracker_click = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_click"].apply(len).mean()
    tracker_internal = this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) >= 0 ) ]["trackers_internal"].apply(len).mean()
    
    plot_data.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR: tracker_first,
                      AFTER_STR: tracker_click,
                      ADDITIONAL_STR: tracker_internal,
                     })
    
    tracker_first = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_first"].apply(len)
    tracker_click = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_click"].apply(len)
    tracker_internal = this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) >= 0 ) ]["trackers_internal"].apply(len)
      
    plot_data_error.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR: (round(np.mean(tracker_first) - 1.95*np.std(tracker_first)/np.sqrt(len(tracker_first)),2),
                                   round(np.mean(tracker_first) + 1.95*np.std(tracker_first)/np.sqrt(len(tracker_first)),2)),
                      AFTER_STR: (round(np.mean(tracker_click) - 1.95*np.std(tracker_click)/np.sqrt(len(tracker_click)),2),
                                   round(np.mean(tracker_click) + 1.95*np.std(tracker_click)/np.sqrt(len(tracker_click)),2)),
                      ADDITIONAL_STR: (round(np.mean(tracker_internal) - 1.95*np.std(tracker_click)/np.sqrt(len(tracker_internal)),2),
                                   round(np.mean(tracker_internal) + 1.95*np.std(tracker_click)/np.sqrt(len(tracker_internal)),2)),
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.sort_values(by="country")
plot_df.index = plot_df["country"]
del plot_df["country"]

THIS_PLOT_ARGS = PLOT_ARGS.copy()
THIS_PLOT_ARGS["figsize"] = (4.25,2.25)


plot = fastplot.plot( plot_df [[BEFORE_STR, AFTER_STR, ADDITIONAL_STR] ]  ,None, 
               mode='bars_multi', ylabel = 'Average Trackers Per Website',
               legend = True, legend_loc='upper left', legend_ncol=3, legend_fontsize=9,
               grid_axis="y", yticks=(np.arange(0,21,2) ,None),
               ylim = (0,21), **THIS_PLOT_ARGS)

for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch("\\\\\\\\\\")
    p.set_edgecolor ("red")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):2*len(plot_df)] + [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")

for p in plot.gca().patches[2*len(plot_df):] + [plot.gca().get_legend().get_patches()[2]]:
    p.set_hatch("----")
    p.set_edgecolor ("green")
    p.set_facecolor ("white")
    
#plot.gca().patches[5].set_hatch("XXXX")
#plot.gca().patches[11].set_hatch("++++")
#plot.gca().patches[17].set_hatch("....")

for i, tup in enumerate(plot_df.itertuples()):
    #print(i, tup.Index)
    this_d = None
    for d in plot_data_error:
        if d["country"] == tup.Index:
            this_d = d
    #print (tup.Index, this_d)
    plot.gca().plot( (i-0.2,i-0.2), (this_d[BEFORE_STR]), linestyle="-", color="black"  )
    plot.gca().plot( (i,i), (this_d[AFTER_STR]), linestyle="-", color="black" )
    plot.gca().plot( (i+0.2,i+0.2), (this_d[ADDITIONAL_STR]), linestyle="-", color="black" )
    
plot.savefig("plots/cookieaccept_trackers_per_website.pdf")
plot.show()

C:\Users\zhang\AppData\Local\Temp\ipykernel_876\1515215486.py:73: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [47]:
plot_data = []
plot_data_error = []

for country in similarweb_websites.keys():
    this_grouped = grouped[grouped[country] == True]
    tracker_first_wo = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==False)]["trackers_first"].apply(len).mean()
    tracker_click_wo = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==False)]["trackers_click"].apply(len).mean()
    tracker_first_w = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==True)]["trackers_first"].apply(len).mean()
    tracker_click_w = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==True)]["trackers_click"].apply(len).mean()
     
    plot_data.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR +  " (w/o Banner)": tracker_click_wo, # Use the "Click" visits, to take into account all 
                      BEFORE_STR + " (w/ Banner)": tracker_first_w,
                      AFTER_STR +  " (w/ Banner)": tracker_click_w,
                     })

    tracker_first_wo = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==False)]["trackers_first"].apply(len)
    tracker_click_wo = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==False)]["trackers_click"].apply(len)
    tracker_first_w = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==True)]["trackers_first"].apply(len)
    tracker_click_w = this_grouped[(this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 )) & ( this_grouped["has_found_banner"]==True)]["trackers_click"].apply(len)
     
    plot_data_error.append({"country": COUNTRY_TRAD[country],
                      BEFORE_STR +  " (w/o Banner)": (round(np.mean(tracker_click_wo) - 1.95*np.std(tracker_click_wo)/np.sqrt(len(tracker_click_wo)),2),
                                   round(np.mean(tracker_click_wo) + 1.95*np.std(tracker_click_wo)/np.sqrt(len(tracker_click_wo)),2)),
                      BEFORE_STR + " (w/ Banner)": (round(np.mean(tracker_first_w) - 1.95*np.std(tracker_click)/np.sqrt(len(tracker_first_w)),2),
                                   round(np.mean(tracker_first_w) + 1.95*np.std(tracker_first_w)/np.sqrt(len(tracker_first_w)),2)),
                      AFTER_STR +  " (w/ Banner)": (round(np.mean(tracker_click_w) - 1.95*np.std(tracker_click_w)/np.sqrt(len(tracker_click_w)),2),
                                   round(np.mean(tracker_click_w) + 1.95*np.std(tracker_click_w)/np.sqrt(len(tracker_click_w)),2)),
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.sort_values(by="country")
plot_df.index = plot_df["country"]
del plot_df["country"]

THIS_PLOT_ARGS = PLOT_ARGS.copy()
THIS_PLOT_ARGS["figsize"] = (4.25,2.25)

plot = fastplot.plot( plot_df [[BEFORE_STR +  " (w/o Banner)", BEFORE_STR + " (w/ Banner)", AFTER_STR +  " (w/ Banner)"] ]  ,None, 
               mode='bars_multi', ylabel = 'Average Trackers Per Website',
               legend = True, legend_loc='upper left', legend_ncol=2, legend_fontsize=8.75, legend_alpha = 1,
               grid_axis="y", yticks=(np.arange(0,21,2) ,None),
               ylim = (0,21), **THIS_PLOT_ARGS)

for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch(r'|\|\|\|\\')
    p.set_edgecolor ("indianred")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):2*len(plot_df)] + [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch(r'-\-\-\-\\')
    p.set_edgecolor ("darkred")
    p.set_facecolor ("white")

for p in plot.gca().patches[2*len(plot_df):] + [plot.gca().get_legend().get_patches()[2]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")
    

for i, tup in enumerate(plot_df.itertuples()):
    #print(i, tup.Index)
    this_d = None
    for d in plot_data_error:
        if d["country"] == tup.Index:
            this_d = d
    #print (tup.Index, this_d)
    plot.gca().plot( (i-0.2,i-0.2), (this_d[BEFORE_STR +  " (w/o Banner)"]), linestyle="-", color="black"  )
    plot.gca().plot( (i,i), (this_d[BEFORE_STR +  " (w/ Banner)"]), linestyle="-", color="black" )
    plot.gca().plot( (i+0.2,i+0.2), (this_d[AFTER_STR +  " (w/ Banner)"]), linestyle="-", color="black" )
    
# plot.savefig("../plots/cookieaccept_trackers_per_website_separate.pdf")
plot.savefig("plots/cookieaccept_trackers_per_website_separate.pdf")
plot.show()


c:\Users\zhang\miniconda3\Lib\site-packages\fastplot\__init__.py:57: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)
C:\Users\zhang\AppData\Local\Temp\ipykernel_876\2726574755.py:73: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [48]:
tracker_first = Counter()
tracker_click = Counter()
tot = len(this_grouped.index)

for row in this_grouped.itertuples():
    for t in row.trackers_first:
        tracker_first[t] +=1
    for t in row.trackers_click:
        tracker_click[t] +=1        

plot_data = []
for t in tracker_first:
    plot_data.append({
        "tracker": t,
        BEFORE_STR: tracker_first[t]/tot*100,
        AFTER_STR: tracker_click[t]/tot*100,
    })
    
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.sort_values(by = BEFORE_STR, ascending=False)


plot_list = [(BEFORE_STR,   (np.arange(len(plot_df[BEFORE_STR].values ) ) +1 ,   plot_df[BEFORE_STR].values  ) ),
             (AFTER_STR, (np.arange(len(plot_df[AFTER_STR].values) ) +1 , plot_df[AFTER_STR].values ) ),
            ]

plot = fastplot.plot( plot_list ,None, 
               mode='line_multi', ylabel = 'Pervasiveness [\\%]',
               legend = True, legend_loc='lower left', legend_ncol=1,
               yscale = "log", xscale = "log", xlabel = "Tracker Rank",
               xlim = (0.85, None), cycler = CYCLER_LINESPOINTS, plot_args = {"markersize": 4, "markevery": slice(0, 10, 1)},
               ylim = (0.01,100), **PLOT_ARGS)

plot.savefig("plots/cookieaccept_tracker_pervasiveness_log.pdf")
plot.show()

print("Trackers on >= 1% of websites: {} -> {}".format( len([e for e in plot_list[0][1][1] if e > 1 ]), len([e for e in plot_list[1][1][1] if e >1 ]) ))

Trackers on >= 1% of websites: 111 -> 123


C:\Users\zhang\AppData\Local\Temp\ipykernel_876\3252227757.py:35: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [33]:
plot_data = []
plot_data_error = []

for category in categories.keys():
    this_grouped = grouped[grouped[category] == True]
    this_grouped = this_grouped [ this_grouped[["us"]].any(axis=1) ] 
    tracker_first = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_first"].apply(len).mean()
    tracker_click = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_click"].apply(len).mean()
    tracker_internal = this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) >= 0 ) ]["trackers_internal"].apply(len).mean()
    
    plot_data.append({"category": category,
                     BEFORE_STR: tracker_first,
                      AFTER_STR: tracker_click,
                      ADDITIONAL_STR: tracker_internal,
                     })
    
    tracker_first = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_first"].apply(len)
    tracker_click = this_grouped[this_grouped["trackers_click"].apply(lambda s: len(s) >= 0 ) ]["trackers_click"].apply(len)
    tracker_internal = this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) >= 0 ) ]["trackers_internal"].apply(len)
      
    plot_data_error.append({"category": category,
                      BEFORE_STR: (round(np.mean(tracker_first) - 1.95*np.std(tracker_first)/np.sqrt(len(tracker_first)),2),
                                   round(np.mean(tracker_first) + 1.95*np.std(tracker_first)/np.sqrt(len(tracker_first)),2)),
                      AFTER_STR: (round(np.mean(tracker_click) - 1.95*np.std(tracker_click)/np.sqrt(len(tracker_click)),2),
                                   round(np.mean(tracker_click) + 1.95*np.std(tracker_click)/np.sqrt(len(tracker_click)),2)),
                      ADDITIONAL_STR: (round(np.mean(tracker_internal) - 1.95*np.std(tracker_click)/np.sqrt(len(tracker_internal)),2),
                                   round(np.mean(tracker_internal) + 1.95*np.std(tracker_click)/np.sqrt(len(tracker_internal)),2)),
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df.index = plot_df["category"]
plot_df = plot_df.sort_values(by = BEFORE_STR, ascending=False)

del plot_df["category"]
plot = fastplot.plot( plot_df[[BEFORE_STR, AFTER_STR, ADDITIONAL_STR] ] ,None, 
               mode='bars_multi', ylabel = '\\begin{center}Average Trackers\\\\Per Website\\end{center}', xticks_rotate = 30,  xticks_fontsize = "small",
               legend = True, legend_loc='upper right', legend_ncol=3,
               grid_axis="y",yticks = (np.arange(0,55,5), None),
               ylim = (0,50), **PLOT_ARGS_LARGE)

for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch("\\\\\\\\\\")
    p.set_edgecolor ("red")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):2*len(plot_df)] + [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")

for p in plot.gca().patches[2*len(plot_df):] + [plot.gca().get_legend().get_patches()[2]]:
    p.set_hatch("----")
    p.set_edgecolor ("green")
    p.set_facecolor ("white")
    

for i, tup in enumerate(plot_df.itertuples()):
    for d in plot_data_error:
        if d["category"] == tup.Index:
            this_d = d
    plot.gca().plot( (i-0.2,i-0.2), (this_d[BEFORE_STR]), linestyle="-", color="black"  )
    plot.gca().plot( (i,i), (this_d[AFTER_STR]), linestyle="-", color="black" )
    plot.gca().plot( (i+0.2,i+0.2), (this_d[ADDITIONAL_STR]), linestyle="-", color="black" )
plot.savefig("plots/cookieaccept_trackers_per_website_category.pdf")
plot.show()
plot_df["Increase"] = plot_df[AFTER_STR] - plot_df[BEFORE_STR]



C:\Users\zhang\AppData\Local\Temp\ipykernel_876\702965879.py:64: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [34]:
plot_data = []
plot_data_error = []

for category in categories.keys():
    this_grouped = grouped[grouped[category] == True]
    this_grouped = this_grouped [ this_grouped[["us"]].any(axis=1) ] 
    tracker_internal = this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) >= 0 ) ]["trackers_internal"].apply(len).mean()
    
    plot_data.append({"category": category,
                      ADDITIONAL_STR: tracker_internal,
                     })
    
    tracker_internal = this_grouped[this_grouped["trackers_internal"].apply(lambda s: len(s) >= 0 ) ]["trackers_internal"].apply(len)
      
    plot_data_error.append({"category": category,
                      ADDITIONAL_STR: (round(np.mean(tracker_internal) - 1.95*np.std(tracker_click)/np.sqrt(len(tracker_internal)),2),
                                   round(np.mean(tracker_internal) + 1.95*np.std(tracker_click)/np.sqrt(len(tracker_internal)),2)),
                     })
    
plot_df = pd.DataFrame(plot_data)
plot_df.index = plot_df["category"]
plot_df = plot_df.sort_values(by = ADDITIONAL_STR, ascending=False)

del plot_df["category"]
plot = fastplot.plot( plot_df[[ADDITIONAL_STR] ] ,None, 
               mode='bars_multi', ylabel = '\\begin{center}Average Trackers\\\\Per Website\\end{center}', xticks_rotate = 30,  xticks_fontsize = "small",
               legend = True, legend_loc='upper right', legend_ncol=3,
               grid_axis="y",yticks = (np.arange(0,55,5), None),
               ylim = (0,50), **PLOT_ARGS_LARGE)



for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch("/////")
    p.set_edgecolor ("red")
    p.set_facecolor ("white")
    

for i, tup in enumerate(plot_df.itertuples()):
    for d in plot_data_error:
        if d["category"] == tup.Index:
            this_d = d
    plot.gca().plot( (i+0.2,i+0.2), (this_d[ADDITIONAL_STR]), linestyle="-", color="black" )
plot.savefig("plots/cookieaccept_trackers_per_website_category.pdf")
plot.show()



C:\Users\zhang\AppData\Local\Temp\ipykernel_876\3662004351.py:45: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()


In [39]:
occ = Counter()
this_grouped = grouped [ grouped[["us"]].any(axis=1) ] 

for s in this_grouped["tp_first"].values:
    for t in s:
        occ[t] +=1
        
monitor_tp = occ.most_common(15)
print(monitor_tp)
plot_data = []


for tp, _ in monitor_tp:
    tot = len(this_grouped.index)
    tp_first = len(this_grouped[this_grouped["tp_first"].apply(lambda s: tp in s) ].index)
    tp_click = len(this_grouped[this_grouped["tp_click"].apply(lambda s: tp in s) ].index)  
    
    tracker_first = len(this_grouped[this_grouped["trackers_first"].apply(lambda s: tp in s) ].index)
    tracker_click = len(this_grouped[this_grouped["trackers_internal"].apply(lambda s: tp in s) ].index) 
    # print(tracker_first)
    # print(tracker_click)
    plot_data.append({"tp": tp,
                  BEFORE_STR : tp_first/tot*100,
                  AFTER_STR: tp_click/tot*100,
                  BEFORE_STR + " (Tracker)": tracker_first/tot*100,
                  AFTER_STR +" (Tracker)": tracker_click/tot*100,    
                 })
    
plot_df = pd.DataFrame(plot_data)
plot_df["tp"] = plot_df["tp"].replace("google.it","google.*")
plot_df.index = plot_df["tp"]
del plot_df["tp"]
plot = fastplot.plot( plot_df [[BEFORE_STR, AFTER_STR] ] ,None, 
               mode='bars_multi', ylabel = 'Pervasiveness [\\%]', xticks_rotate = 45, 
               legend = True, legend_loc='upper right', legend_ncol=1, legend_fontsize = "small", xticks_fontsize="small",
                     legend_alpha=0.6,
               grid_axis="y", yticks = ([0,10,20,30,40,50,60,70,80], None),
               ylim = (0,80), **PLOT_ARGS)

for p in plot.gca().patches[:len(plot_df)] + [plot.gca().get_legend().get_patches()[0]]:
    p.set_hatch("\\\\\\\\\\")
    p.set_edgecolor ("red")
    p.set_facecolor ("white")
for p in plot.gca().patches[len(plot_df):]+ [plot.gca().get_legend().get_patches()[1]]:
    p.set_hatch("/////")
    p.set_edgecolor ("blue")
    p.set_facecolor ("white")
    
N=this_grouped["sld"].nunique()
# err = plot_df[[BEFORE_STR, AFTER_STR]].applymap(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*N/100,N,0.05) ])
err = plot_df[[BEFORE_STR, AFTER_STR]].map(lambda v: [ e*100 for e in statsmodels.stats.proportion.proportion_confint(v*N/100,N,0.05) ])
for i, tup in enumerate(err.itertuples()):
    plot.gca().plot( (i-0.15,i-0.15), (tup[1]), linestyle="-", color="black"  )
    plot.gca().plot( (i+0.15,i+0.15), (tup[2]), linestyle="-", color="black" )

plot.savefig("plots/cookieaccept_top_tp.pdf")
plot.show()
plot_df["increase"] = plot_df[AFTER_STR] - plot_df[BEFORE_STR]


[('googletagmanager.com', 1277), ('google.com', 1137), ('doubleclick.net', 1100), ('google-analytics.com', 1037), ('googleapis.com', 752), ('gstatic.com', 750), ('facebook.net', 628), ('facebook.com', 616), ('adnxs.com', 524), ('linkedin.com', 420), ('googlesyndication.com', 402), ('bing.com', 399), ('rubiconproject.com', 375), ('amazon-adsystem.com', 371), ('casalemedia.com', 352)]


C:\Users\zhang\AppData\Local\Temp\ipykernel_876\915634906.py:57: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot.show()
